In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [4]:
final_merged_df = pd.read_csv('parkingformatted.csv')

In [5]:
final_merged_df['start_time'] = pd.to_datetime(final_merged_df.start_time,format='%Y/%m/%d').astype('datetime64[s]')

In [6]:
df_tst_merged_5min= final_merged_df[['start_time','no_of_cars']]
df_tst_merged_5min = df_tst_merged_5min.set_index('start_time')

In [7]:
df_tst_merged_5min.head(30)

,no_of_cars
start_time,
2018-08-11 00:00:00,2
2018-08-11 00:05:00,0
2018-08-11 00:10:00,1
2018-08-11 00:15:00,3
2018-08-11 00:20:00,3
2018-08-11 00:25:00,2
2018-08-11 00:30:00,2
2018-08-11 00:35:00,3
2018-08-11 00:40:00,1


In [8]:
df_tst_merged_5min.plot(figsize=(15,9))

In [9]:
from pandas import read_csv
from pandas import datetime
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
# load dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
print(df_tst_merged_5min.head())
# transform scale
X = df_tst_merged_5min.values
X = X.reshape(len(X), 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(X)
scaled_X = scaler.transform(X)
scaled_series = Series(scaled_X[:, 0])
print(scaled_series.head())
# invert transform
inverted_X = scaler.inverse_transform(scaled_X)
inverted_series = Series(inverted_X[:, 0])
print(inverted_series.head())

                     no_of_cars
start_time                     
2018-08-11 00:00:00           2
2018-08-11 00:05:00           0
2018-08-11 00:10:00           1
2018-08-11 00:15:00           3
2018-08-11 00:20:00           3
0   -0.789474
1   -1.000000
2   -0.894737
3   -0.684211
4   -0.684211
dtype: float64
0    2.0
1    0.0
2    1.0
3    3.0
4    3.0
dtype: float64


/Users/mano/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [38]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy

# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
		model.reset_states()
	return model

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

# load dataset
# series1 = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

# transform data to be stationary
raw_values = df_tst_merged_5min.values
diff_values = difference(raw_values, 1)

# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]

# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

# fit the model
lstm_model = fit_lstm(train_scaled, 1, 50, 400)
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
# lstm_model.predict(train_reshaped, batch_size=1)

# # walk-forward validation on the test data
# predictions = list()
# for i in range(len(test_scaled)):
# 	# make one-step forecast
# 	X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
# 	yhat = forecast_lstm(lstm_model, 1, X)
# 	# invert scaling
# 	yhat = invert_scale(scaler, X, yhat)
# 	# invert differencing
# 	yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
# 	# store forecast
# 	predictions.append(yhat)
# 	expected = raw_values[len(train) + i + 1]
# 	print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

# # report performance
# rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
# print('Test RMSE: %.3f' % rmse)
# # line plot of observed vs predicted
# pyplot.plot(raw_values[-12:])
# pyplot.plot(predictions)
# pyplot.show()

Epoch 1/1
19283/19283 [==============================] - 138s 7ms/step - loss: 0.0110
Epoch 1/1
19283/19283 [==============================] - 139s 7ms/step - loss: 0.0094
Epoch 1/1
10343/19283 [===============>..............] - ETA: 1:05 - loss: 0.0094

KeyboardInterrupt: 

In [10]:
df_tst_merged_5min.tail()

,no_of_cars
start_time,
2019-04-11 20:55:00,3
2019-04-11 21:00:00,1
2019-04-11 21:05:00,1
2019-04-11 21:10:00,1
2019-04-11 21:15:00,3


In [11]:
df_tst_merged_5min['(t-1)'] = df_tst_merged_5min.no_of_cars.shift(-2)
df_tst_merged_5min['(t-2)'] = df_tst_merged_5min.no_of_cars.shift(-3)
df_tst_merged_5min['(t-3)'] = df_tst_merged_5min.no_of_cars.shift(-4)
df_tst_merged_5min.head()

,no_of_cars,(t-1),(t-2),(t-3)
start_time,,,,
2018-08-11 00:00:00,2,1.0,3.0,3.0
2018-08-11 00:05:00,0,3.0,3.0,2.0
2018-08-11 00:10:00,1,3.0,2.0,2.0
2018-08-11 00:15:00,3,2.0,2.0,3.0
2018-08-11 00:20:00,3,2.0,3.0,1.0


In [12]:
df_tst_merged_5min.tail()

,no_of_cars,(t-1),(t-2),(t-3)
start_time,,,,
2019-04-11 20:55:00,3,1.0,1.0,3.0
2019-04-11 21:00:00,1,1.0,3.0,NaN
2019-04-11 21:05:00,1,3.0,NaN,NaN
2019-04-11 21:10:00,1,NaN,NaN,NaN
2019-04-11 21:15:00,3,NaN,NaN,NaN


In [13]:
df_tst_merged_5min.dropna(inplace=True)

In [14]:
df_tst_merged_5min.tail()

,no_of_cars,(t-1),(t-2),(t-3)
start_time,,,,
2019-04-11 20:35:00,0,3.0,6.0,3.0
2019-04-11 20:40:00,0,6.0,3.0,1.0
2019-04-11 20:45:00,3,3.0,1.0,1.0
2019-04-11 20:50:00,6,1.0,1.0,1.0
2019-04-11 20:55:00,3,1.0,1.0,3.0


In [15]:
test_set = df_tst_merged_5min[-30:]
train_set = df_tst_merged_5min[:-30]
X_test_set = test_set.drop(['(t-3)'], axis=1)
y_test_set = test_set['(t-3)']
X_train_set = train_set.drop(['(t-3)'], axis=1)
y_train_set = train_set['(t-3)']

In [16]:
y_test_set

start_time
2019-04-11 18:30:00    6.0
2019-04-11 18:35:00    2.0
2019-04-11 18:40:00    2.0
2019-04-11 18:45:00    3.0
2019-04-11 18:50:00    1.0
2019-04-11 18:55:00    0.0
2019-04-11 19:00:00    3.0
2019-04-11 19:05:00    4.0
2019-04-11 19:10:00    2.0
2019-04-11 19:15:00    4.0
2019-04-11 19:20:00    0.0
2019-04-11 19:25:00    0.0
2019-04-11 19:30:00    3.0
2019-04-11 19:35:00    2.0
2019-04-11 19:40:00    2.0
2019-04-11 19:45:00    5.0
2019-04-11 19:50:00    4.0
2019-04-11 19:55:00    4.0
2019-04-11 20:00:00    5.0
2019-04-11 20:05:00    2.0
2019-04-11 20:10:00    0.0
2019-04-11 20:15:00    0.0
2019-04-11 20:20:00    0.0
2019-04-11 20:25:00    3.0
2019-04-11 20:30:00    6.0
2019-04-11 20:35:00    3.0
2019-04-11 20:40:00    1.0
2019-04-11 20:45:00    1.0
2019-04-11 20:50:00    1.0
2019-04-11 20:55:00    3.0
Name: (t-3), dtype: float64

In [17]:
import warnings
warnings.filterwarnings('ignore')
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [18]:
X = array(X_train_set)
y = array(y_train_set)
X

array([[2., 1., 3.],
       [0., 3., 3.],
       [1., 3., 2.],
       ...,
       [2., 1., 4.],
       [1., 4., 1.],
       [1., 1., 1.]])

In [19]:
y

array([3., 2., 2., ..., 1., 1., 1.])

In [20]:
X = X.reshape((X.shape[0], X.shape[1],1))
X

array([[[2.],
        [1.],
        [3.]],

       [[0.],
        [3.],
        [3.]],

       [[1.],
        [3.],
        [2.]],

       ...,

       [[2.],
        [1.],
        [4.]],

       [[1.],
        [4.],
        [1.]],

       [[1.],
        [1.],
        [1.]]])

In [35]:
model = Sequential()
model.add(LSTM(256, activation = 'relu', input_shape=(X.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,scoring="accuracy")
grid_result = grid.fit(X, y, epochs=30, validation_split=0.10)
print((grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print((mean, stdev, param))

# model.fit(X, y, epochs=30, validation_split=0.10)

TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x1c2f1cc780>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

In [98]:
x_test = array(X_test_set)
x_input = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
prediction = model.predict(x_input, verbose=0)
print(prediction)

[[1.5904529 ]
 [3.3453658 ]
 [2.200326  ]
 [1.90239   ]
 [2.6461463 ]
 [1.5720267 ]
 [0.88638264]
 [2.3810704 ]
 [2.729725  ]
 [1.9743059 ]
 [2.8892503 ]
 [1.1241379 ]
 [0.77479464]
 [2.5019598 ]
 [1.8777657 ]
 [1.7803657 ]
 [3.1401544 ]
 [3.0547194 ]
 [3.2727964 ]
 [4.4740577 ]
 [2.201961  ]
 [1.0889504 ]
 [1.0877085 ]
 [0.77479464]
 [2.3000846 ]
 [3.0864015 ]
 [2.2965713 ]
 [1.5500402 ]
 [1.8229153 ]
 [1.4719658 ]]


In [99]:
prediction=prediction.round()

In [100]:
prediction

array([[2.],
       [3.],
       [2.],
       [2.],
       [3.],
       [2.],
       [1.],
       [2.],
       [3.],
       [2.],
       [3.],
       [1.],
       [1.],
       [3.],
       [2.],
       [2.],
       [3.],
       [3.],
       [3.],
       [4.],
       [2.],
       [1.],
       [1.],
       [1.],
       [2.],
       [3.],
       [2.],
       [2.],
       [2.],
       [1.]], dtype=float32)

In [85]:
y_test_set.values

array([6., 2., 2., 3., 1., 0., 3., 4., 2., 4., 0., 0., 3., 2., 2., 5., 4.,
       4., 5., 2., 0., 0., 0., 3., 6., 3., 1., 1., 1., 3.])

In [101]:
sns.set_style("ticks")
plt.figure(figsize=(25,5))
plt.plot(y_test_set.values, label = "Actual")
plt.plot(prediction, label = "Predicted")
plt.legend(loc='best')
plt.show()

In [93]:
y_test_set

start_time
2019-04-11 18:30:00    6.0
2019-04-11 18:35:00    2.0
2019-04-11 18:40:00    2.0
2019-04-11 18:45:00    3.0
2019-04-11 18:50:00    1.0
2019-04-11 18:55:00    0.0
2019-04-11 19:00:00    3.0
2019-04-11 19:05:00    4.0
2019-04-11 19:10:00    2.0
2019-04-11 19:15:00    4.0
2019-04-11 19:20:00    0.0
2019-04-11 19:25:00    0.0
2019-04-11 19:30:00    3.0
2019-04-11 19:35:00    2.0
2019-04-11 19:40:00    2.0
2019-04-11 19:45:00    5.0
2019-04-11 19:50:00    4.0
2019-04-11 19:55:00    4.0
2019-04-11 20:00:00    5.0
2019-04-11 20:05:00    2.0
2019-04-11 20:10:00    0.0
2019-04-11 20:15:00    0.0
2019-04-11 20:20:00    0.0
2019-04-11 20:25:00    3.0
2019-04-11 20:30:00    6.0
2019-04-11 20:35:00    3.0
2019-04-11 20:40:00    1.0
2019-04-11 20:45:00    1.0
2019-04-11 20:50:00    1.0
2019-04-11 20:55:00    3.0
Name: (t-3), dtype: float64

In [102]:
print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test_set.values, prediction)))

The R2 score on the Test set is:	-0.113


In [103]:
model.evaluate(x_input, y_test_set, batch_size=1)

30/30 [==============================] - 0s 1ms/step


3.5399659251018116

In [107]:
import math
math.sqrt(mean_squared_error(y_test_set.values, prediction))

1.879716290649558

In [ ]:
# grid search lstm for airline passengers
from math import sqrt
from numpy import array
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# transform list into supervised learning format
def series_to_supervised(data, n_in=1, n_out=1):
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	agg.dropna(inplace=True)
	return agg.values

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# difference dataset
def difference(data, order):
	return [data[i] - data[i - order] for i in range(order, len(data))]

# fit a model
def model_fit(train, config):
	# unpack config
	n_input, n_nodes, n_epochs, n_batch, n_diff = config
	# prepare data
	if n_diff > 0:
		train = difference(train, n_diff)
	# transform series into supervised format
	data = series_to_supervised(train, n_in=n_input)
	# separate inputs and outputs
	train_x, train_y = data[:, :-1], data[:, -1]
	# reshape input data into [samples, timesteps, features]
	n_features = 1
	train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
	# define model
	model = Sequential()
	model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
	model.add(Dense(n_nodes, activation='relu'))
	model.add(Dense(1))
	model.compile(loss='mse', optimizer='adam')
	# fit model
	model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
	return model

# forecast with the fit model
def model_predict(model, history, config):
	# unpack config
	n_input, _, _, _, n_diff = config
	# prepare data
	correction = 0.0
	if n_diff > 0:
		correction = history[-n_diff]
		history = difference(history, n_diff)
	# reshape sample into [samples, timesteps, features]
	x_input = array(history[-n_input:]).reshape((1, n_input, 1))
	# forecast
	yhat = model.predict(x_input, verbose=0)
	return correction + yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# fit model
	model = model_fit(train, cfg)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = model_predict(model, history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	print(' > %.3f' % error)
	return error

# score a model, return None on failure
def repeat_evaluate(data, config, n_test, n_repeats=10):
	# convert config to a key
	key = str(config)
	# fit and evaluate the model n times
	scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
	# summarize score
	result = mean(scores)
	print('> Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test):
	# evaluate configs
	scores = scores = [repeat_evaluate(data, cfg, n_test) for cfg in cfg_list]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a list of configs to try
def model_configs():
	# define scope of configs
	n_input = [12]
	n_nodes = [100]
	n_epochs = [10]
	n_batch = [1]
	n_diff = [0]
	# create configs
	configs = list()
	for i in n_input:
		for j in n_nodes:
			for k in n_epochs:
				for l in n_batch:
					for m in n_diff:
						cfg = [i, j, k, l, m]
						configs.append(cfg)
	print('Total configs: %d' % len(configs))
	return configs

# define dataset
series = read_csv('parkingformatted.csv', header=0, index_col=0)
data = series.no_of_cars.values
# data split
n_test = 12
# model configs
cfg_list = model_configs()
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 10 configs
for cfg, error in scores[:3]:
	print(cfg, error)

Total configs: 1


In [46]:
series = read_csv('parkingformatted.csv', header=0, index_col=0)
data = series.no_of_cars.values


In [44]:
series

,no_of_cars,summary,temperature
start_time,,,
2018-08-11 00:00:00,2,Clear,87.880000
2018-08-11 00:05:00,0,Clear,87.880000
2018-08-11 00:10:00,1,Clear,87.880000
2018-08-11 00:15:00,3,Clear,87.880000
2018-08-11 00:20:00,3,Clear,87.880000
2018-08-11 00:25:00,2,Clear,87.880000
2018-08-11 00:30:00,2,Clear,87.880000
2018-08-11 00:35:00,3,Clear,87.880000
2018-08-11 00:40:00,1,Clear,87.880000


In [47]:
train, test = train_test_split(data, n_test)

In [48]:
def difference(data, order):
	return [data[i] - data[i - order] for i in range(order, len(data))]


In [49]:
difference(train, 12)

[-1,
 2,
 0,
 1,
 -2,
 2,
 0,
 -1,
 3,
 3,
 0,
 -1,
 1,
 2,
 4,
 -1,
 1,
 -1,
 1,
 2,
 -1,
 -3,
 -2,
 -1,
 3,
 2,
 1,
 2,
 3,
 0,
 -1,
 -1,
 0,
 2,
 3,
 1,
 -2,
 -3,
 -4,
 -3,
 -3,
 -2,
 1,
 0,
 2,
 1,
 1,
 2,
 2,
 2,
 3,
 2,
 3,
 4,
 2,
 2,
 0,
 0,
 -1,
 -1,
 -1,
 -1,
 -2,
 -2,
 -3,
 -3,
 -3,
 -3,
 -3,
 -2,
 0,
 0,
 0,
 0,
 1,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -4,
 -3,
 -3,
 -3,
 -3,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 4,
 5,
 5,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -2,
 -1,
 -1,
 -1,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -1,
 -2,
 -2,
 -2,
 -3,
 -3,
 -2,
 -2,
 -2,
 -2,
 -2,
 -3,
 -2,
 1,
 -1,
 -1,
 2,
 2,
 1,
 0,
 0,
 1,
 -1,
 2,
 0,
 -2,
 0,
 0,
 

In [43]:
for i in range(12, len(data)):
    print(data[i])

[1 'Clear' 83.91]
[2 'Clear' 83.91]
[1 'Clear' 83.91]
[4 'Clear' 83.91]
[1 'Clear' 83.91]
[4 'Clear' 83.91]
[2 'Clear' 83.91]
[2 'Clear' 83.91]
[4 'Clear' 83.91]
[5 'Clear' 83.91]
[3 'Clear' 83.91]
[3 'Clear' 83.91]
[2 'Clear' 80.84]
[4 'Clear' 80.84]
[5 'Clear' 80.84]
[3 'Clear' 80.84]
[2 'Clear' 80.84]
[3 'Clear' 80.84]
[3 'Clear' 80.84]
[4 'Clear' 80.84]
[3 'Clear' 80.84]
[2 'Clear' 80.84]
[1 'Clear' 80.84]
[2 'Clear' 80.84]
[5 'Clear' 78.09]
[6 'Clear' 78.09]
[6 'Clear' 78.09]
[5 'Clear' 78.09]
[5 'Clear' 78.09]
[3 'Clear' 78.09]
[2 'Clear' 78.09]
[3 'Clear' 78.09]
[3 'Clear' 78.09]
[4 'Clear' 78.09]
[4 'Clear' 78.09]
[3 'Clear' 78.09]
[3 'Clear' 76.85]
[3 'Clear' 76.85]
[2 'Clear' 76.85]
[2 'Clear' 76.85]
[2 'Clear' 76.85]
[1 'Clear' 76.85]
[3 'Clear' 76.85]
[3 'Clear' 76.85]
[5 'Clear' 76.85]
[5 'Clear' 76.85]
[5 'Clear' 76.85]
[5 'Clear' 76.85]
[5 'Clear' 73.54]
[5 'Clear' 73.54]
[5 'Clear' 73.54]
[4 'Clear' 73.54]
[5 'Clear' 73.54]
[5 'Clear' 73.54]
[5 'Clear' 73.54]
[5 'Clear'

[2 'Mostly Cloudy' 88.09]
[2 'Mostly Cloudy' 88.09]
[2 'Mostly Cloudy' 88.09]
[3 'Mostly Cloudy' 88.09]
[3 'Mostly Cloudy' 88.09]
[4 'Mostly Cloudy' 88.09]
[4 'Mostly Cloudy' 88.09]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Mostly Cloudy' 89.35]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Partly Cloudy' 90.23]
[4 'Mostly Cloudy' 89.11]
[4 'Mostly Cloudy' 89.11]
[4 'Mostly Cloudy' 89.11]
[4 'Mostly Cloudy' 89.11]
[4 'Mostly Cloudy' 89.11]
[4 'Mostly Cloudy' 89.11]
[4 'Mostly Cloudy' 89.11]
[4 'Mostly C

[4 'Clear' 77.95]
[3 'Clear' 77.95]
[3 'Clear' 77.95]
[3 'Clear' 77.95]
[3 'Clear' 77.95]
[2 'Clear' 77.95]
[2 'Clear' 77.95]
[3 'Clear' 77.95]
[3 'Clear' 76.2]
[3 'Clear' 76.2]
[3 'Clear' 76.2]
[3 'Clear' 76.2]
[3 'Clear' 76.2]
[8 'Clear' 76.2]
[5 'Clear' 76.2]
[2 'Clear' 76.2]
[3 'Clear' 76.2]
[1 'Clear' 76.2]
[3 'Clear' 76.2]
[1 'Clear' 76.2]
[0 'Partly Cloudy' 74.03]
[0 'Partly Cloudy' 74.03]
[0 'Partly Cloudy' 74.03]
[0 'Partly Cloudy' 74.03]
[2 'Partly Cloudy' 74.03]
[3 'Partly Cloudy' 74.03]
[4 'Partly Cloudy' 74.03]
[2 'Partly Cloudy' 74.03]
[2 'Partly Cloudy' 74.03]
[1 'Partly Cloudy' 74.03]
[1 'Partly Cloudy' 74.03]
[1 'Partly Cloudy' 74.03]
[1 'Partly Cloudy' 72.13]
[0 'Partly Cloudy' 72.13]
[0 'Partly Cloudy' 72.13]
[3 'Partly Cloudy' 72.13]
[2 'Partly Cloudy' 72.13]
[2 'Partly Cloudy' 72.13]
[1 'Partly Cloudy' 72.13]
[3 'Partly Cloudy' 72.13]
[2 'Partly Cloudy' 72.13]
[2 'Partly Cloudy' 72.13]
[0 'Partly Cloudy' 72.13]
[2 'Partly Cloudy' 72.13]
[3 'Clear' 70.3]
[2 'Clear' 

[0 'Mostly Cloudy' 76.86]
[0 'Mostly Cloudy' 76.86]
[0 'Mostly Cloudy' 76.86]
[2 'Mostly Cloudy' 76.86]
[2 'Mostly Cloudy' 76.86]
[4 'Mostly Cloudy' 76.86]
[6 'Mostly Cloudy' 76.86]
[1 'Mostly Cloudy' 76.86]
[0 'Mostly Cloudy' 76.86]
[1 'Mostly Cloudy' 76.47]
[1 'Mostly Cloudy' 76.47]
[1 'Mostly Cloudy' 76.47]
[0 'Mostly Cloudy' 76.47]
[4 'Mostly Cloudy' 76.47]
[3 'Mostly Cloudy' 76.47]
[2 'Mostly Cloudy' 76.47]
[3 'Mostly Cloudy' 76.47]
[3 'Mostly Cloudy' 76.47]
[2 'Mostly Cloudy' 76.47]
[1 'Mostly Cloudy' 76.47]
[2 'Mostly Cloudy' 76.47]
[2 'Partly Cloudy' 76.61]
[3 'Partly Cloudy' 76.61]
[2 'Partly Cloudy' 76.61]
[2 'Partly Cloudy' 76.61]
[3 'Partly Cloudy' 76.61]
[2 'Partly Cloudy' 76.61]
[2 'Partly Cloudy' 76.61]
[0 'Partly Cloudy' 76.61]
[0 'Partly Cloudy' 76.61]
[0 'Partly Cloudy' 76.61]
[0 'Partly Cloudy' 76.61]
[1 'Partly Cloudy' 76.61]
[3 'Clear' 79.23]
[2 'Clear' 79.23]
[2 'Clear' 79.23]
[3 'Clear' 79.23]
[0 'Clear' 79.23]
[2 'Clear' 79.23]
[5 'Clear' 79.23]
[3 'Clear' 79.23

[6 'Clear' 74.24]
[6 'Clear' 74.24]
[6 'Clear' 74.24]
[6 'Clear' 74.24]
[5 'Clear' 74.24]
[5 'Clear' 74.24]
[5 'Clear' 74.24]
[5 'Clear' 74.24]
[4 'Clear' 74.24]
[5 'Clear' 74.24]
[5 'Clear' 74.24]
[5 'Clear' 77.63]
[5 'Clear' 77.63]
[1 'Clear' 77.63]
[1 'Clear' 77.63]
[1 'Clear' 77.63]
[1 'Clear' 77.63]
[1 'Clear' 77.63]
[2 'Clear' 77.63]
[2 'Clear' 77.63]
[3 'Clear' 77.63]
[2 'Clear' 77.63]
[2 'Clear' 77.63]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Clear' 80.66]
[2 'Humid' 83.84]
[2 'Humid' 83.84]
[2 'Humid' 83.84]
[1 'Humid' 83.84]
[0 'Humid' 83.84]
[0 'Humid' 83.84]
[0 'Humid' 83.84]
[0 'Humid' 83.84]
[0 'Humid' 83.84]
[0 'Humid' 83.84]
[0 'Humid' 83.84]
[0 'Humid' 83.84]
[0 'Humid' 85.48]
[0 'Humid' 85.48]
[0 'Humid' 85.48]
[0 'Humid' 85.48]
[0 'Humid' 85.48]
[0 'Humid' 85.48]
[1 'Humid' 85.48]
[1 'Humid' 85.48]
[1 'Humid'

...

[3 'Overcast' 39.5]
[2 'Overcast' 39.5]
[1 'Overcast' 39.5]
[1 'Overcast' 39.5]
[1 'Overcast' 39.5]
[2 'Overcast' 39.5]
[1 'Overcast' 39.34]
[0 'Overcast' 39.34]
[0 'Overcast' 39.34]
[0 'Overcast' 39.34]
[1 'Overcast' 39.34]
[1 'Overcast' 39.34]
[1 'Overcast' 39.34]
[4 'Overcast' 39.34]
[1 'Overcast' 39.34]
[1 'Overcast' 39.34]
[1 'Overcast' 39.34]
[0 'Overcast' 39.34]
[1 'Overcast' 40.28]
[3 'Overcast' 40.28]
[5 'Overcast' 40.28]
[4 'Overcast' 40.28]
[4 'Overcast' 40.28]
[2 'Overcast' 40.28]
[2 'Overcast' 40.28]
[2 'Overcast' 40.28]
[4 'Overcast' 40.28]
[4 'Overcast' 40.28]
[6 'Overcast' 40.28]
[3 'Overcast' 40.28]
[2 'Overcast' 39.94]
[6 'Overcast' 39.94]
[3 'Overcast' 39.94]
[3 'Overcast' 39.94]
[1 'Overcast' 39.94]
[4 'Overcast' 39.94]
[4 'Overcast' 39.94]
[1 'Overcast' 39.94]
[0 'Overcast' 39.94]
[4 'Overcast' 39.94]
[4 'Overcast' 39.94]
[3 'Overcast' 39.94]
[2 'Overcast' 39.5]
[2 'Overcast' 39.5]
[4 'Overcast' 39.5]
[3 'Overcast' 39.5]
[4 'Overcast' 39.5]
[3 'Overcast' 39.5]
[8 '

[1 'Partly Cloudy' 58.63]
[4 'Partly Cloudy' 58.63]
[6 'Partly Cloudy' 58.63]
[4 'Partly Cloudy' 58.63]
[3 'Partly Cloudy' 58.63]
[4 'Partly Cloudy' 58.63]
[3 'Mostly Cloudy' 54.2]
[3 'Mostly Cloudy' 54.2]
[2 'Mostly Cloudy' 54.2]
[2 'Mostly Cloudy' 54.2]
[2 'Mostly Cloudy' 54.2]
[3 'Mostly Cloudy' 54.2]
[3 'Mostly Cloudy' 54.2]
[3 'Mostly Cloudy' 54.2]
[1 'Mostly Cloudy' 54.2]
[4 'Mostly Cloudy' 54.2]
[3 'Mostly Cloudy' 54.2]
[5 'Mostly Cloudy' 54.2]
[2 'Overcast' 50.17]
[1 'Overcast' 50.17]
[1 'Overcast' 50.17]
[3 'Overcast' 50.17]
[5 'Overcast' 50.17]
[2 'Overcast' 50.17]
[3 'Overcast' 50.17]
[1 'Overcast' 50.17]
[3 'Overcast' 50.17]
[3 'Overcast' 50.17]
[3 'Overcast' 50.17]
[1 'Overcast' 50.17]
[1 'Overcast' 48.4]
[2 'Overcast' 48.4]
[2 'Overcast' 48.4]
[1 'Overcast' 48.4]
[1 'Overcast' 48.4]
[3 'Overcast' 48.4]
[4 'Overcast' 48.4]
[3 'Overcast' 48.4]
[1 'Overcast' 48.4]
[1 'Overcast' 48.4]
[0 'Overcast' 48.4]
[0 'Overcast' 48.4]
[0 'Overcast' 46.41]
[0 'Overcast' 46.41]
[0 'Overca

[4 'Clear' 63.12]
[2 'Clear' 63.12]
[2 'Clear' 63.12]
[5 'Clear' 63.12]
[1 'Clear' 63.12]
[3 'Clear' 63.12]
[3 'Clear' 63.12]
[0 'Clear' 63.12]
[2 'Clear' 63.12]
[0 'Clear' 63.12]
[0 'Clear' 63.12]
[0 'Clear' 61.16]
[1 'Clear' 61.16]
[1 'Clear' 61.16]
[3 'Clear' 61.16]
[2 'Clear' 61.16]
[1 'Clear' 61.16]
[1 'Clear' 61.16]
[2 'Clear' 61.16]
[5 'Clear' 61.16]
[4 'Clear' 61.16]
[5 'Clear' 61.16]
[3 'Clear' 61.16]
[2 'Clear' 59.35]
[4 'Clear' 59.35]
[2 'Clear' 59.35]
[0 'Clear' 59.35]
[3 'Clear' 59.35]
[0 'Clear' 59.35]
[0 'Clear' 59.35]
[2 'Clear' 59.35]
[1 'Clear' 59.35]
[2 'Clear' 59.35]
[0 'Clear' 59.35]
[1 'Clear' 59.35]
[0 'Clear' 56.8]
[1 'Clear' 56.8]
[2 'Clear' 56.8]
[5 'Clear' 56.8]
[4 'Clear' 56.8]
[0 'Clear' 56.8]
[1 'Clear' 56.8]
[1 'Clear' 56.8]
[1 'Clear' 56.8]
[3 'Clear' 56.8]
[0 'Clear' 56.8]
[0 'Clear' 56.8]
[1 'Clear' 55.44]
[1 'Clear' 55.44]
[3 'Clear' 55.44]
[3 'Clear' 55.44]
[2 'Clear' 55.44]
[0 'Clear' 55.44]
[0 'Clear' 55.44]
[1 'Clear' 55.44]
[2 'Clear' 55.44]
[7 '

[2 'Clear' 54.22]
[3 'Clear' 54.22]
[2 'Clear' 54.22]
[2 'Clear' 55.55]
[0 'Clear' 55.55]
[0 'Clear' 55.55]
[0 'Clear' 55.55]
[0 'Clear' 55.55]
[4 'Clear' 55.55]
[1 'Clear' 55.55]
[0 'Clear' 55.55]
[3 'Clear' 55.55]
[5 'Clear' 55.55]
[3 'Clear' 55.55]
[2 'Clear' 55.55]
[1 'Partly Cloudy' 55.45]
[2 'Partly Cloudy' 55.45]
[4 'Partly Cloudy' 55.45]
[3 'Partly Cloudy' 55.45]
[4 'Partly Cloudy' 55.45]
[2 'Partly Cloudy' 55.45]
[2 'Partly Cloudy' 55.45]
[1 'Partly Cloudy' 55.45]
[3 'Partly Cloudy' 55.45]
[2 'Partly Cloudy' 55.45]
[2 'Partly Cloudy' 55.45]
[2 'Partly Cloudy' 55.45]
[1 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.76]
[4 'Partly Cloudy' 54.76]
[2 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.76]
[2 'Partly Cloudy' 54.76]
[1 'Partly Cloudy' 54.55]
[2 'Partly Cloudy' 54.55]
[1 'Partly Cloudy' 54.55]
[1 'Partly Cloudy' 54.55]
[4

[5 'Clear' 63.85]
[4 'Clear' 63.85]
[1 'Clear' 63.85]
[1 'Clear' 63.85]
[1 'Clear' 62.03]
[0 'Clear' 62.03]
[0 'Clear' 62.03]
[0 'Clear' 62.03]
[1 'Clear' 62.03]
[0 'Clear' 62.03]
[3 'Clear' 62.03]
[2 'Clear' 62.03]
[1 'Clear' 62.03]
[2 'Clear' 62.03]
[2 'Clear' 62.03]
[1 'Clear' 62.03]
[0 'Clear' 60.84]
[3 'Clear' 60.84]
[0 'Clear' 60.84]
[1 'Clear' 60.84]
[4 'Clear' 60.84]
[1 'Clear' 60.84]
[1 'Clear' 60.84]
[1 'Clear' 60.84]
[2 'Clear' 60.84]
[1 'Clear' 60.84]
[0 'Clear' 60.84]
[0 'Clear' 60.84]
[0 'Clear' 59.58]
[0 'Clear' 59.58]
[1 'Clear' 59.58]
[2 'Clear' 59.58]
[2 'Clear' 59.58]
[1 'Clear' 59.58]
[0 'Clear' 59.58]
[1 'Clear' 59.58]
[0 'Clear' 59.58]
[0 'Clear' 59.58]
[0 'Clear' 59.58]
[0 'Clear' 59.58]
[0 'Clear' 60.94]
[0 'Clear' 60.94]
[0 'Clear' 60.94]
[0 'Clear' 60.94]
[1 'Clear' 60.94]
[1 'Clear' 60.94]
[2 'Clear' 60.94]
[1 'Clear' 60.94]
[0 'Clear' 60.94]
[1 'Clear' 60.94]
[1 'Clear' 60.94]
[1 'Clear' 60.94]
[1 'Clear' 58.41]
[0 'Clear' 58.41]
[0 'Clear' 58.41]
[0 'Clear'

In [ ]:
data[i - order]